# Project: AML End-to-End Threshold Tuning Methodology

Author: Sam MA  
Date: 12 Mar 2022  
  
Introduction:   

This project aims to help clients determine the optimal parameters for transaction monitoring rules of anti-money laundering(AML) system, so that clients could detect as many suspicious transactions as possible with less labor cost, while also meeting regulatory requirements.  

In this case, we will use rule TM003 as an example.
In TM003, if the amount of a single transaction exceeds the parameter (e.g. HKD40,000), or the cumulative total amount of each user on the day exceeds the parameter (e.g. HKD50,000), then the transaction would be regarded as an alert. We are aiming at tuning out the optimal parameters with an acceptable number of alerts and relatively high true positive rate. 

## Library and Data Preparation

In [1]:
import pandas as pd
import numpy as np

In [2]:
AML_data = pd.read_excel('AML.xlsx',sheet_name='Raw Data')
AML_data

,User_id,IsPrivateBank,Type,Risk,Account,Date,Amount,FraudReview
0,6011,non-PB,interbank,H,20317308,2020-02-12,6907451,N
1,8004,PB,company,M,37440750,2020-08-23,2103574,Y
2,1413,non-PB,company,M,20484061,2020-08-16,9081372,Y
3,6934,PB,individual,L,51716580,2020-01-20,2629238,Y
4,9837,PB,interbank,H,20758651,2020-04-01,4415771,N
...,...,...,...,...,...,...,...,...
10860,3886,PB,individual,L,74742760,2020-03-07,5101806,N
10861,3739,PB,individual,L,59049255,2020-03-18,6308954,N
10862,2758,non-PB,individual,L,15454452,2020-11-25,5849764,N
10863,6020,non-PB,individual,L,13459035,2020-03-20,9712754,N


Please note that the dataset was generated by myself, not the production data in reality.  

Key fields in the dataset:  
  -- [IsPrivateBank]: 'PB'(private bank), 'non-PB'(non-private bank)  
  -- [Type]: 'interbank','company','individual'  
  -- [Risk]: 'H','M','L'  
  -- [Account]: Account number  
  -- [Date]: Transaction date  
  -- [Amount]: Transaction amount  
  -- [FraudReview]: Historical reviewed result by client's compliance team, 'Y' refers to fraud/problem case, 'N' refers to normal case

## Create Percentile Table 

To determine the optimal parameters, we need to leverage percentile to run simulation.  
Here we would apply 90 to 99 percentile to run the simmulation.

In [3]:
Percentile = pd.DataFrame(columns=['Rule','var1_per','var2_per'])

for u in range(90,100):
    for v in range(90,100):
        Percentile = Percentile.append(pd.DataFrame({'Rule':'TM003',
                                                     'var1_per':[u/100],
                                                     'var2_per':[v/100]})
                                       ,ignore_index=True)
Percentile

,Rule,var1_per,var2_per
0,TM003,0.90,0.90
1,TM003,0.90,0.91
2,TM003,0.90,0.92
3,TM003,0.90,0.93
4,TM003,0.90,0.94
...,...,...,...
95,TM003,0.99,0.95
96,TM003,0.99,0.96
97,TM003,0.99,0.97
98,TM003,0.99,0.98


## Create Master Table

Theoretically, users could be divided into 18 groups from three dimensions, however, 7 groups were observed in the dataset.  
We need to determine the optimal patameters for these 7 groups for TM003.

In [4]:
Rule_TM003 = pd.DataFrame(columns=['Rule','ID','var1','var2'])

for i in (AML_data['IsPrivateBank']+'&'+AML_data['Type']+'&'+AML_data['Risk']).unique():
    Rule_TM003 = Rule_TM003.append(pd.DataFrame({'Rule':'TM003',
                                                   'ID': [i],
                                                   'var1':'Trade Amount',
                                                   'var2':'Daily Cumulative Amount'})
                                  ,ignore_index=True)

Rule_TM003

,Rule,ID,var1,var2
0,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount
1,TM003,PB&company&M,Trade Amount,Daily Cumulative Amount
2,TM003,non-PB&company&M,Trade Amount,Daily Cumulative Amount
3,TM003,PB&individual&L,Trade Amount,Daily Cumulative Amount
4,TM003,PB&interbank&H,Trade Amount,Daily Cumulative Amount
5,TM003,non-PB&individual&L,Trade Amount,Daily Cumulative Amount
6,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount


Now we would combine the master table with percentile:

In [5]:
Percentile_TM003 = pd.merge(Rule_TM003,Percentile,on=['Rule'])
Percentile_TM003

,Rule,ID,var1,var2,var1_per,var2_per
0,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.90
1,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.91
2,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.92
3,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.93
4,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.94
...,...,...,...,...,...,...
695,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.95
696,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.96
697,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.97
698,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.98


## Calculate var1_value

Here we would define some functions to calculate value for var1_per under each ID (user group)

In [6]:
def get_var1_value(x,u):
    return np.percentile(AML_data.loc[(AML_data['IsPrivateBank']+'&'+AML_data['Type']+'&'+AML_data['Risk']==x),:]['Amount'],u*100)

#get_var1_value('non-PB&individual&L',0.99) #an example

In [7]:
Percentile_TM003['var1_value'] = Percentile_TM003.apply(lambda x: get_var1_value(x.ID,x.var1_per), axis=1)
Percentile_TM003

,Rule,ID,var1,var2,var1_per,var2_per,var1_value
0,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.90,9032409.50
1,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.91,9032409.50
2,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.92,9032409.50
3,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.93,9032409.50
4,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.94,9032409.50
...,...,...,...,...,...,...,...
695,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.95,7215628.52
696,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.96,7215628.52
697,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.97,7215628.52
698,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.98,7215628.52


## Calculate var2_value

Here we would define some functions to calculate value for var2_per under each ID (user group)

The first step is to add cumulative amount for each user_id and date in the raw data:

In [8]:
def Cal_Accm_Amount(x,y):
    return AML_data.loc[(AML_data['User_id']==x)&(AML_data['Date']==y),:]['Amount'].sum()

# Cal_Accm_Amount(1528,'2020-07-22') # an examle

In [9]:
AML_data['Accm_Amount'] = AML_data.apply(lambda x: Cal_Accm_Amount(x.User_id,x.Date),axis=1)
AML_data

,User_id,IsPrivateBank,Type,Risk,Account,Date,Amount,FraudReview,Accm_Amount
0,6011,non-PB,interbank,H,20317308,2020-02-12,6907451,N,6907451
1,8004,PB,company,M,37440750,2020-08-23,2103574,Y,2103574
2,1413,non-PB,company,M,20484061,2020-08-16,9081372,Y,9081372
3,6934,PB,individual,L,51716580,2020-01-20,2629238,Y,2629238
4,9837,PB,interbank,H,20758651,2020-04-01,4415771,N,4415771
...,...,...,...,...,...,...,...,...,...
10860,3886,PB,individual,L,74742760,2020-03-07,5101806,N,5101806
10861,3739,PB,individual,L,59049255,2020-03-18,6308954,N,6308954
10862,2758,non-PB,individual,L,15454452,2020-11-25,5849764,N,5849764
10863,6020,non-PB,individual,L,13459035,2020-03-20,9712754,N,9712754


The second step is to calculate value for var2_per in the master table:

In [10]:
def get_var2_value(x,u):
    return np.percentile(AML_data.loc[(AML_data['IsPrivateBank']+'&'+AML_data['Type']+'&'+AML_data['Risk']==x),:]['Accm_Amount'],u*100)    
#get_var2_value('non-PB&individual&L',0.91)

In [11]:
Percentile_TM003['var2_value'] = Percentile_TM003.apply(lambda x: get_var2_value(x.ID,x.var2_per), axis=1)
Percentile_TM003

,Rule,ID,var1,var2,var1_per,var2_per,var1_value,var2_value
0,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.90,9032409.50,9038862.50
1,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.91,9032409.50,9129717.26
2,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.92,9032409.50,9209910.08
3,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.93,9032409.50,9324598.20
4,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.94,9032409.50,9410928.48
...,...,...,...,...,...,...,...,...
695,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.95,7215628.52,7154614.60
696,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.96,7215628.52,7169868.08
697,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.97,7215628.52,7185121.56
698,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.98,7215628.52,7200375.04


## Update Calculation Result

The first step is to define some functions for calculation:

In [12]:
# to calculate total count for the ID(user group)
def CalTotalCount(x):
    return len(
        AML_data.loc[(AML_data['IsPrivateBank']+'&'+AML_data['Type']+'&'+AML_data['Risk']==x),:]
        )

# to calculate the number of alerts with the given parameters
def CalPositiveCount(x,y,z):
    return len(
        AML_data.loc[(AML_data['IsPrivateBank']+'&'+AML_data['Type']+'&'+AML_data['Risk']==x)
                     &((AML_data['Amount']>=y)|(AML_data['Accm_Amount']>=z)),:]
        )

# to calculate the number of true positive alerts with the given parameters
def CalTruePositiveCount(x,y,z):
    return len(
        AML_data.loc[(AML_data['IsPrivateBank']+'&'+AML_data['Type']+'&'+AML_data['Risk']==x)
                     &((AML_data['Amount']>=y)|(AML_data['Accm_Amount']>=z))&(AML_data['FraudReview']=='Y'),:]
        )

The following step is to calculate the number of transaction('TotalCount'), number of alerts('TotalPositiveCount') and number of true positive alerts('TotalTruePositiveCount'):

In [13]:
Percentile_TM003['TotalCount'] = Percentile_TM003.apply(lambda x: CalTotalCount(x.ID), axis=1)

Percentile_TM003['TotalPositiveCount'] = Percentile_TM003.apply(lambda x: CalPositiveCount(x.ID,x.var1_value,x.var2_value), axis=1)

Percentile_TM003['TotalTruePositiveCount'] = Percentile_TM003.apply(lambda x: CalTruePositiveCount(x.ID,x.var1_value,x.var2_value), axis=1)

In [14]:
Percentile_TM003

,Rule,ID,var1,var2,var1_per,var2_per,var1_value,var2_value,TotalCount,TotalPositiveCount,TotalTruePositiveCount
0,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.90,9032409.50,9038862.50,1660,168,78
1,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.91,9032409.50,9129717.26,1660,168,78
2,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.92,9032409.50,9209910.08,1660,168,78
3,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.93,9032409.50,9324598.20,1660,168,78
4,TM003,non-PB&interbank&H,Trade Amount,Daily Cumulative Amount,0.90,0.94,9032409.50,9410928.48,1660,166,78
...,...,...,...,...,...,...,...,...,...,...,...
695,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.95,7215628.52,7154614.60,5,1,1
696,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.96,7215628.52,7169868.08,5,1,1
697,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.97,7215628.52,7185121.56,5,1,1
698,TM003,non-PB&interbank&M,Trade Amount,Daily Cumulative Amount,0.99,0.98,7215628.52,7200375.04,5,1,1


## Result Export

Table Percentile_TM003 would be exported to excel for further visulation and presentation:  
(Please refer to AML.xlsx)

In [15]:
Percentile_TM003.to_excel('Percentile_TM003_Result.xlsx',sheet_name='Percentile_TM003_Result')

END